# G336 c8 c9 workflow

## Prepare data

Rename fields and sources

In [2]:
import casatasks as tasks
from pathlib import Path

In [3]:
data = Path('../uvdata')
visc8 = data / 'G336.018-0.827.c8.ms'
visc9 = data / 'G336.018-0.827.c9.ms'
field = 'G336.018-0.827'

_ = tasks.listobs(f'{visc8}')
_ = tasks.listobs(f'{visc8}')


In [3]:
tasks.vishead(f'{visc8}', mode='put', hdkey='field', hdindex='0', hdvalue=field)
tasks.vishead(f'{visc9}', mode='put', hdkey='field', hdindex='0', hdvalue=field)

for i in range(4):
    tasks.vishead(f'{visc8}', mode='put', hdkey='source_name', hdindex=f'{i}', hdvalue=field)
    tasks.vishead(f'{visc9}', mode='put', hdkey='source_name', hdindex=f'{i}', hdvalue=field)
    
_ = tasks.listobs(f'{visc9}')
_ = tasks.listobs(f'{visc8}')

Change phase centers

In [4]:
from casatools import msmetadata

metadata = msmetadata()
metadata.open(f'{visc9}')
phasecenter = metadata.phasecenter()
print(phasecenter)
metadata.close()

{'m0': {'unit': 'rad', 'value': -1.9410090954645292}, 'm1': {'unit': 'rad', 'value': -0.8513663731352502}, 'refer': 'ICRS', 'type': 'direction'}


True

In [5]:
metadata.open(f'{visc8}')
print(metadata.phasecenter())
metadata.close()

{'m0': {'unit': 'rad', 'value': -1.9410023323151402}, 'm1': {'unit': 'rad', 'value': -0.8513723799767234}, 'refer': 'ICRS', 'type': 'direction'}


True

In [6]:
from astropy.coordinates import SkyCoord

phasecenter_coord = SkyCoord(phasecenter['m0']['value'], phasecenter['m1']['value'], unit=(phasecenter['m0']['unit'], phasecenter['m1']['unit']), frame=phasecenter['refer'].lower())
print(phasecenter_coord)

<SkyCoord (ICRS): (ra, dec) in deg
    (248.78837083, -48.7797)>


In [7]:
new_phasecenter = phasecenter_coord.to_string('hmsdms')
new_phasecenter = 'ICRS ' + new_phasecenter
print(new_phasecenter)

ICRS 16h35m09.20900001s -48d46m46.92000004s


In [8]:
new_phasecenter = 'ICRS 16h35m09.2090s -48d46m46.920s'
outputvis = visc8.with_suffix('.shifted.ms')
tasks.phaseshift(f'{visc8}', outputvis=f'{outputvis}', phasecenter=new_phasecenter)

_ = tasks.listobs(f'{outputvis}')

In [9]:
new_phasecenter = 'ICRS 16h35m09.2090s -48d46m46.920s'
outputvis = visc8.with_suffix('.shifted.fixvis.ms')
tasks.fixvis(vis=f'{visc8}', outputvis=f'{outputvis}', phasecenter=new_phasecenter)

_ = tasks.listobs(f'{outputvis}')

2024-02-02 06:43:26	WARN	fixvis::::casa	**ALERT:** Deprecated in CASA 5.9/6.3. Please use task **phaseshift** instead.


Now do selfcal

## After selfcal

Apply selfcal table and split corrected.

In [4]:
visc8_fix = visc8.with_suffix('.shifted.fixvis.ms')
concatvis = data / 'G336.018-0.827.c8c9.ms'
tasks.concat(vis=[f'{visc9}', f'{visc8_fix}'], concatvis=f'{concatvis}')
_ = tasks.listobs(f'{concatvis}')

In [5]:
caltable = f'../manual_selfcal/{field}.3.phase.cal'
tasks.applycal(vis=f'{concatvis}',
               gaintable=[caltable],
               spwmap=['0','0','0','0','4','4','4','4'],
               interp='linear')
visc8c9 = concatvis.with_suffix('.selfcal.ms')


NameError: name 'concat_selfcal' is not defined

In [6]:
tasks.split(f'{concatvis}', outputvis=f'{visc8c9}', datacolumn='corrected')

Get SPW information, cvel and concat result

In [7]:
from casatools import msmetadata
metadata = msmetadata()
metadata.open(f'{visc8c9}')

nspw = metadata.nspw()
print(nspw)

8


In [11]:
for i in range(nspw):
    freqs = metadata.chanfreqs(spw=i, unit='GHz')
    dnu = metadata.chanwidths(spw=i, unit='kHz')
    print(f'SPW {i}: {freqs[0]} -- {freqs[-1]} (dnu={dnu})')

SPW 0: 233.55088282351068 -- 235.42539454226068 (dnu=[488.28125 488.28125 488.28125 ... 488.28125 488.28125 488.28125])
SPW 1: 231.0510092767289 -- 232.9255209954789 (dnu=[488.28125 488.28125 488.28125 ... 488.28125 488.28125 488.28125])
SPW 2: 218.72623924988272 -- 216.85172753113272 (dnu=[-488.28125 -488.28125 -488.28125 ... -488.28125 -488.28125 -488.28125])
SPW 3: 220.92612797106665 -- 219.05161625231665 (dnu=[-488.28125 -488.28125 -488.28125 ... -488.28125 -488.28125 -488.28125])
SPW 4: 218.52795984847307 -- 216.65344812972307 (dnu=[-488.28125 -488.28125 -488.28125 ... -488.28125 -488.28125 -488.28125])
SPW 5: 221.34045984847307 -- 219.46594812972307 (dnu=[-488.28125 -488.28125 -488.28125 ... -488.28125 -488.28125 -488.28125])
SPW 6: 230.98644812972307 -- 232.86095984847307 (dnu=[488.28125 488.28125 488.28125 ... 488.28125 488.28125 488.28125])
SPW 7: 233.54894812972307 -- 235.42345984847307 (dnu=[488.28125 488.28125 488.28125 ... 488.28125 488.28125 488.28125])


In [15]:
pairings = [(0, 7), (1, 6), (2, 4), (3, 5)]
for i, pair in enumerate(pairings):
    freq1 = metadata.chanfreqs(spw=pair[0], unit='GHz')
    freq2 = metadata.chanfreqs(spw=pair[1], unit='GHz')
    freq_low = max(min(freq1), min(freq2))
    freq_high = min(max(freq1), max(freq2))
    dnu = metadata.chanwidths(spw=i, unit='kHz')[0]
    nchan = abs(freq_high*1E9 - freq_low*1E9) / abs(dnu*1E3)
    if dnu > 0:
        start = freq_low
    else:
        start = freq_high
    print(f'SPW {i}: {freq_low} -- {freq_high} (dnu={dnu}, nchan={int(nchan)})')
    outputvis = visc8c9.with_suffix(f'.spw{i}.cvel.ms')
    tasks.cvel2(f'{visc8c9}', outputvis=f'{outputvis}', spw=','.join(map(str, pair)), mode='frequency', start=f'{start}GHz', width=f'{dnu}kHz', nchan=int(nchan))

SPW 0: 233.55088282351068 -- 235.42345984847307 (dnu=488.28125, nchan=3835)
SPW 1: 231.0510092767289 -- 232.86095984847307 (dnu=488.28125, nchan=3706)
SPW 2: 216.85172753113272 -- 218.52795984847307 (dnu=-488.28125, nchan=3432)
SPW 3: 219.46594812972307 -- 220.92612797106665 (dnu=-488.28125, nchan=2990)


In [16]:
vis = [visc8c9.with_suffix(f'.spw{i}.cvel.ms') for i in range(len(pairings))]
concatvis = visc8c9.with_suffix(f'.cvel.ms')
print(vis)
tasks.concat(vis=list(map(str, vis)), concatvis=f'{concatvis}')

[PosixPath('../uvdata/G336.018-0.827.c8c9.selfcal.spw0.cvel.ms'), PosixPath('../uvdata/G336.018-0.827.c8c9.selfcal.spw1.cvel.ms'), PosixPath('../uvdata/G336.018-0.827.c8c9.selfcal.spw2.cvel.ms'), PosixPath('../uvdata/G336.018-0.827.c8c9.selfcal.spw3.cvel.ms')]
